In [ ]:
from langchain_community.document_loaders import TextLoader #vai converter o texto em um formato que o Lang Chain possa trabalhar
from langchain_text_splitters import CharacterTextSplitter #vai dividir o texto em diferentes chunks de caracteres (há diferentes formas de fazer isso. Vale à pena pesquisar)
from langchain_openai import OpenAIEmbeddings #vai converter o texto em embeddings (vetores) que o Lang Chain possa trabalhar utilizando modelos do OpenAI
from langchain_chroma import Chroma #vaii guardar os vectors em vectors databases

In [ ]:
# Vamos agora utilizar as chaves de API do OpenAI. Parta isso vamos utilizar a a biblioteca dotenv
# com .env

from dotenv import load_dotenv

load_dotenv()


In [ ]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')
display(books)

In [ ]:
display(books["tag_description"])

#A tag é uma ótima forma de indentificar o nome do livro após utilizar a descrição para treinar o modelo, pois o que vamos ter de retorno na busca de vetores 
# é a descrição com a tag do livro. Utilizar a descrição para filtrar pelo livro é muito devagar, por isso utilizar a tag é uma melhor opçção

#vamos então extrair as descrições e as tags dos livros

books["tag_description"].to_csv("books_tag_description.csv", index=False, sep = "\n", header=False)


In [ ]:
raw_documents = TextLoader("books_tag_description.csv").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap = 0, separator = "\n") #vamos utilizar chunk_size = 0 para garantir que ele vai fazer a separação baseado no separator 
# e não em chunks de caracteres e utilizar chunk_overlap = 0 para garantir que ele não vai haver overlap entre as descrições.
documents = text_splitter.split_documents(raw_documents)


In [ ]:
#checando abaixo percebe-se que funcionou corretamente a separação das descrições
documents[0]

In [ ]:
# vamos agora criar os embeddings para cada descrição e salvá-los em um banco de dados de vetores

from langchain_huggingface import HuggingFaceEmbeddings
huggingface_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=huggingface_embeddings
)

In [ ]:
#podemos agora dar um query para o banco de dados de vetores para encontrar os livros mais similares a essa query

query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
display(docs)

In [ ]:
#Mas nós queremos que o usuário possa pesquisar por um livro e não por uma descrição

#vamos extrair então os isbn13 di oage_content e utilizar para filtrar o nome do livro com o valor de docs
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]


In [ ]:
def find_book_by_description(query, books_df, db_books, k=10):
    docs = db_books.similarity_search(query, k=k)
    all_codes = [int(doc.page_content.split()[0].strip('"')) for doc in docs]
    return books_df[books_df["isbn13"].isin(all_codes)]

find_book_by_description("A book about games", books, db_books)

# Zero-shot classification

Agora vamos trabalhar com zero-shot classification, onde conseguimos utilizar modelos LLM para prever o tipo de um livro a partir da sua descrição

In [ ]:
# Agora quer nós temos um modelos de busca de livros baseado em descrições, podemos procurar formas de refinar essa busca.


#Vamos fazer uma classificação utilizando somente modelos de ficção e não-ficção. Então vamos simplificar as categorias
# Nós vamos então utilizar de text-classification para diminuir o número de categorias nos nossos dados e utilizar isso
# como um potencial filtro no BookRecommender

# A utilização dos LLMs para esse fim é chamado de zero-shot classification, que praticamente associada uma descrição ou texto a uma categoria

#Primeiro vamos definir as cotegorias que gostaríamos de utilizar para classificar os livros: fiction e non-fiction

#Vamos pegar as categorias com 50 ou mais livros

categories_count = books[["categories"]].value_counts().sort_values(ascending=False)
categories_count = categories_count[categories_count >= 50]
display(categories_count)

#E utilizar elas pra dividir os livros em fiction e non-fiction

category_mapping = {
    "Fiction" : "Fiction",
    "Juvenile Fiction" : "Fiction",
    "Biography & Autobiography" : "Nonfiction",
    "History" : "Nonfiction",
    "Literary Criticism": "Nonfiction",
    "Philosophy": "Nonfiction",
    "Religion": "Nonfiction",
    "Comics & Graphic Novels": "Fiction",
    "Drama": "Fiction",
    "Juvenile Nonfiction": "Nonfiction",
    "Science": "Nonfiction",
    "Poetry": "Fiction"
}

books["simple_categories"] = books["categories"].map(category_mapping)

#Percebe-se que temos um bom conjunto de livros que podem ser utilizados na classificação com LLMS

display(books[~(books["simple_categories"].isna())])




In [ ]:
# vamos utilizar então o modelo da hugging face para nosso zero-shot classification

# from transformers import pipeline

# fictions_catregories = ["Fiction", "Nonfiction"]

# pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0) #device = 0 para utilizar a GPU, caso não tenha GPU, pode-se utilizar device=-1



In [ ]:
# sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]
# pipe(sequence, fictions_catregories)


In [ ]:
# vamos utilizar então o modelo da hugging face para nosso zero-shot classification

import numpy as np
import torch
import os
from transformers import pipeline

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from transformers import pipeline
class BookClassification:
    def __init__(self, model_name="facebook/bart-large-mnli", device=0, batch_size=16):
        self.model = model_name
        self.device = device
        self.categories = ["Fiction", "Nonfiction"]
        self.pipe = self._load_model()
        self.batch_size = batch_size
    
    def _load_model(self):
        try:
            return pipeline("zero-shot-classification", model=self.model, device=self.device)
        except torch.OutOfMemoryError as e:
            print(e)
            return pipeline("zero-shot-classification", model=self.model, device=-1)        

    def classify_batch(self, sequences):
        results = self.pipe(sequences, candidate_labels=self.categories, batch_size=self.batch_size)
        if isinstance(results, dict):
            results = [results]
        return [r["labels"][np.argmax(r["scores"])] for r in results]
    

book_classifier = BookClassification(model_name="facebook/bart-large-mnli", device=0, batch_size=4)

In [ ]:
# E então vamso criar duas listas com os valores reais para livros de ficção e não-ficção
# e utilizar o modelo para classificar os livros

from tqdm import tqdm

# Agrupar descrições
fiction_desc = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[:300]
nonfiction_desc = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)[:300]

# Unir tudo
all_descriptions = list(fiction_desc) + list(nonfiction_desc)
actual_cats = ["Fiction"] * 300 + ["Nonfiction"] * 300

# Fazer predições em batch
predicted_cats = []
batch_size = 4
for i in tqdm(range(0, len(all_descriptions), batch_size)):
    batch = all_descriptions[i:i+batch_size]
    predicted = book_classifier.classify_batch(batch)
    predicted_cats.extend(predicted)

In [ ]:
#Agora podemos comparar os valores reais com os valores previstos e verificar a acurácia do modelo

prediction_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})
prediction_df["correct_prection"] = np.where(prediction_df["actual_categories"] == prediction_df["predicted_categories"],1,0)
prediction_df["correct_prection"].sum() / len(prediction_df) * 100

# o modelo teve uma acurácia em volta de 75%, o que é um bom resultado para um modelo de zero-shot classification que não foi designado especificamente para essas categorias



In [ ]:
missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True)

predicted_cats_missing = []
isbn13_missing = []

batch_size = 4
for i in tqdm(range(0, len(missing_cats), batch_size)):
    batch = missing_cats["description"][i:i+batch_size].tolist()
    predicted = book_classifier.classify_batch(batch)
    predicted_cats_missing.extend(predicted)
    isbn13_missing.extend(missing_cats["isbn13"][i:i+batch_size].tolist())